# Population dynamics

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy.integrate import solve_ivp


def integrate(model, y0, parameters, t_end, n_points=100):
    sol = solve_ivp(
        model,
        args=parameters,
        y0=list(y0.values()),
        t_span=(0, t_end),
        t_eval=np.linspace(0, t_end, n_points),
        method="LSODA",
    )
    return pd.DataFrame(sol.y.T, index=sol.t, columns=y0.keys())

## Bacterial growth (Exponential)

In [ ]:
def exponential_growth(_, y, k):
    return k * y

df = integrate(
    exponential_growth,
    y0={"E. coli": 1.0},
    parameters=(1,),
    t_end=5,
)

ax = df.plot(
    xlabel="time / a.u",
    ylabel="concentration / a.u.",
)

## Bacterial growth (Monod)

In [ ]:
def monod(_, y, mu_max, ks, z):
    """
    mu_max: maximum specific growth rate
    ks: half-saturation constant
    z: cell yield factor
    """
    microbes, nutrients = y
    dmic_dt = mu_max * nutrients / (ks + nutrients) * microbes
    dnut_dt = -z * mu_max * nutrients / (ks + nutrients) * microbes
    return dmic_dt, dnut_dt
    

df = integrate(
    monod,
    y0={"E. coli": 1.0, "Glucose": 1.0},
    parameters=(1, 0.5, 0.1),
    t_end=15,
    n_points=1000,
)

ax = df.plot(
    xlabel="time / a.u",
    ylabel="concentration / a.u.",
)

## Lotka-Volterra model

Lotka (1920) and Volterra (1926) independently from each other created a system of differential equations to understand regular variations in animal populations: 

$$\begin{align}    
  \frac{dx}{dt} &= r_1 \cdot x - C_1 \cdot x \cdot y \\
  \frac{dy}{dt} &= C_2 \cdot x \cdot y - r_2 \cdot y \\
\end{align}$$

Here $x$ describes the dynamic of the prey and $y$ the one of the predators. The parameter $r_1$ describes the growth rate of the prey in absence of predators and $r_2$ the growth dying rate of the predators in absence of the prey. The parameters $C_1$ and $C_2$ are coupling parameters that describe, how much prey has to be hunted in order to create a new predator.

In [ ]:
def lotka_volterra(_, y, r1, r2, C1, C2):
    prey, predator = y
    dxdt = r1 * prey - C1 * prey * predator
    dydt = C2 * prey * predator - r2 * predator
    return dxdt, dydt


df = integrate(
    lotka_volterra,
    y0={"Prey": 1.0, "Predator": 1.0},
    parameters=(1, 1, 0.5, 0.5),
    t_end=15,
    n_points=1000,
)

ax = df.plot(
    xlabel="time / a.u",
    ylabel="concentration / a.u.",
)

## SIR model

- Susceptible (S), Infected (I), Recovered (R)
- Total population (N) = S + I + R

$$\begin{align*}
    \frac{dS}{dt} &= -\beta \frac{S \cdot I}{N}  \\
    \frac{dI}{dt} &= \beta \frac{S \cdot I}{N} - \gamma \cdot I \\
    \frac{dR}{dt} &= \gamma \cdot I \\
\end{align*}$$

Let's start with a literal translation of that description.

In [ ]:
def sir_v1(_, y, beta, gamma):
    s, i, r = y
    n = s + i + r

    dsdt = -beta * s * i / n
    didt = beta * s * i / n - gamma * i
    drdt = gamma * i
    return dsdt, didt, drdt


df = integrate(
    sir_v1,
    y0={"S": 0.9, "I": 0.1, "R": 0},
    parameters=(2, 0.1),
    t_end=10,
)

ax = df.plot(
    xlabel="time / a.u",
    ylabel="concentration / a.u.",
)

There is a lot of duplication in the code above (e.g. `beta * s * i / n` appears twice), let's improve on that.

In [ ]:
def sir_v2(_, y, beta, gamma):
    s, i, r = y
    n = s + i + r

    infection = beta * s * i / n
    recovery = gamma * i

    dsdt = -infection
    didt = infection - recovery
    drdt = recovery
    return dsdt, didt, drdt


df = integrate(
    sir_v2,
    y0={"S": 0.9, "I": 0.1, "R": 0},
    parameters=(2, 0.1),
    t_end=10,
)

ax = df.plot()